In [1]:
import numpy as np
import simpy
from scipy.stats import expon

In [2]:
def simulate(lam_dt, lam_st, T): 
    # single server queue: 
    # arrival intensity=lan_dt, service time rate=lam_st, terminal time=T

    def arrivals(): # homogenous poisson process
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        rqt = server.request()
        yield rqt # Enter the server
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st) # Finish service
        server.release(rqt)
        X.append(env.now - t0) # time inside the system

    X = [] # List to store times in the system
    env = simpy.Environment()
    server = simpy.Resource(capacity=1, env=env) # change the capacity: multiple servers shared line
    env.process(arrivals())
    env.run()
    return np.max(X) # Return the maximum time in the system

def run_simulations(n, lam_dt, lam_st, T):
    simulations = [simulate(lam_dt, lam_st, T) for i in range(n)]
    return simulations

In [3]:
run_simulations(10, 1, 1, 10)

[5.87181971157138,
 8.548963555105074,
 2.362438771483716,
 5.1478277107309,
 3.52544335301042,
 1.287467288984928,
 2.893265002165812,
 2.7591210989016073,
 4.830526363325475,
 0.922843975010494]

In [ ]:
# spend some time shopping, then get into the line

def simulate(lam_dt, lam_st, lam_shop, T):
    # lam_shop

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())

            else:
                break
    
    def service():
        t0 = env.now
        shop_dt = expon.rvs(scale=1/lam_shop) # shopping time also follow a exponetial time
        yield env.timeout(shop_dt)
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.Resource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [ ]:
# starbucks
# arrive -> request to the server -> wait in line -> drink order -> request to barista
# register & barista shared queue

def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        rqt_barista = barista.request()
        yield rqt_barista
        barista_dt = expon.rvs(scale=1/lam_st)
        yield env.timeout(barista_dt)
        barista.release(rqt_barista)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.Resource(capacity=1, env=env)
    barista = simpy.Resource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)

In [ ]:
# McDonald
# n1 registers, n2 baristas
# shared queue to the registers
# shared queue to the baristas

def simulate(n1, n2, lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        rqt_barista = barista.request()
        yield rqt_barista
        barista_dt = expon.rvs(scale=1/lam_st)
        yield env.timeout(barista_dt)
        barista.release(rqt_barista)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.Resource(capacity=n1, env=env)
    barista = simpy.Resource(capacity=n2, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)

In [ ]:
## suppose probability is 50% that a person orders a drip coffee

def simulate(n1, n2, lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        if uniform.rvs() < 0.5:
            rqt_barista = barista.request()
            yield rqt_barista
            barista_dt = expon.rvs(scale=1/lam_st)
            yield env.timeout(barista_dt)
            barista.release(rqt_barista)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.Resource(capacity=n1, env=env)
    barista = simpy.Resource(capacity=n2, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [ ]:
# seperate queue of register
# n1 registers with seperate queue
# n2 baristas shared queue

def simulate(n1, n2, lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        line_length = [server.count + len(server.queue) for server in servers]
        ind = np.argmin(line_length) # customer go the the shortiet line
        rqt = servers[ind].request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        servers[ind].release(rqt)
        rqt_barista = barista.request()
        yield rqt_barista
        barista_dt = expon.rvs(scale=1/lam_st)
        yield env.timeout(barista_dt)
        barista.release(rqt_barista)
        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    servers = [simpy.Resource(capacity=1, env=env) for i in range(n1)]
    barista = simpy.Resource(capacity=n2, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [ ]:
# 2 baristas, shared queue, one is experienced, one not

def simulate(n1, n2, lam_dt, lam_st, lam_st1, lam_st2, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        rqt = server.request()
        yield rqt
        st = expon.rvs(scale=1/lam_st)
        yield env.timeout(st)
        server.release(rqt)
        rqt1 = barista1.request()
        rqt2 = barista2.request()
        results = yield simpy.AnyOf(env, [rqt1,rqt2]) # yield the first rqt
        # what if barista 1/2 both not busy?
        if rqt1 in results: # 1 not busy
            if rqt2 in results: # both not busy
                barista2.release(rqt2)
            else: # 2 is busy
                rqt2.cancel()
            barista_dt = expon.rvs(scale=1/lam_st1)
            yield env.timeout(barista_dt)
            barista1.release(rqt1)
        else: # 2 not busy
            rqt1.cancel()
            barista_dt = expon.rvs(scale=1/lam_st2)
            yield env.timeout(barista_dt)
            barista2.release(rqt2)

        X.append(env.now - t0)

    X = []
    env = simpy.Environment()
    server = simpy.Resource(capacity=n1, env=env)
    barista1 = simpy.Resource(capacity=1, env=env)
    barista2 = simpy.Resource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)

def run_simulations(n, n1, n2, lam_dt, lam_st, lam_st1, lam_st2, T):
    simulations = [simulate(n1, n2, lam_dt, lam_st, lam_st1, lam_st2, T) for i in range(n)]
    return simulations

run_simulations(10, 1, 1, 1, 1, 2, 1, 10)

In [4]:
def simulate(lam_dt, lam_st, T):

    def arrivals():
        while True:
            dt = expon.rvs(scale=1/lam_dt)
            if env.now + dt < T:
                yield env.timeout(dt)
                env.process(service())
            else:
                break
    
    def service():
        t0 = env.now
        kitchen_process = env.process(kitchen_pr())
        bar_process = env.process(bar_pr())
        yield simpy.AllOf(env, [kitchen_process, bar_process])
        # only continues when both the kitchen and bar processes are complete
        X.append(env.now - t0)


    def kitchen_pr():
        rqt = kitchen.request()
        yield rqt
        yield env.timeout(expon.rvs(scale=1/lam_st))
        kitchen.release(rqt)

    def bar_pr():
        rqt = bar.request()
        yield rqt
        yield env.timeout(expon.rvs(scale=1/lam_st))
        bar.release(rqt)
    


    X = []
    env = simpy.Environment()
    kitchen = simpy.Resource(capacity=1, env=env)
    bar = simpy.Resource(capacity=1, env=env)
    env.process(arrivals())
    env.run()
    return np.max(X)


In [5]:
simulate(1, 1, 10)

4.520464271302229